In [1]:
import pandas as pd
import numpy as np 


df = pd.read_csv('matches2.csv')

In [2]:
df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [3]:
new_df = df[['team1', 'team2', 'winner', 'toss_decision', 'toss_winner']]

In [4]:
new_df.dropna(inplace=True)
#drop Nan values
#returns new DataFrame object the inplace parameter is not set to True

C:\Users\KD\AppData\Local\Temp\ipykernel_21820\862249462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


In [5]:
X = new_df[['team1', 'team2', 'toss_decision', 'toss_winner']]
y = new_df[['winner']]

In [6]:
all_teams = {}  #dictionary
cnt = 0
for i in range(len(df)):
    if df.loc[i]['team1'] not in all_teams:
        all_teams[df.loc[i]['team1']] = cnt
        cnt += 1    
        
    if df.loc[i]['team2'] not in all_teams:
        all_teams[df.loc[i]['team2']] = cnt
        cnt += 1
print(all_teams)

{'Sunrisers Hyderabad': 0, 'Royal Challengers Bangalore': 1, 'Mumbai Indians': 2, 'Rising Pune Supergiant': 3, 'Gujarat Lions': 4, 'Kolkata Knight Riders': 5, 'Kings XI Punjab': 6, 'Delhi Daredevils': 7, 'Chennai Super Kings': 8, 'Rajasthan Royals': 9, 'Deccan Chargers': 10, 'Kochi Tuskers Kerala': 11, 'Pune Warriors': 12, 'Rising Pune Supergiants': 13, 'Delhi Capitals': 14}


In [7]:
encoded_teams = {w:k for k,w in all_teams.items()}

print(encoded_teams)

{0: 'Sunrisers Hyderabad', 1: 'Royal Challengers Bangalore', 2: 'Mumbai Indians', 3: 'Rising Pune Supergiant', 4: 'Gujarat Lions', 5: 'Kolkata Knight Riders', 6: 'Kings XI Punjab', 7: 'Delhi Daredevils', 8: 'Chennai Super Kings', 9: 'Rajasthan Royals', 10: 'Deccan Chargers', 11: 'Kochi Tuskers Kerala', 12: 'Pune Warriors', 13: 'Rising Pune Supergiants', 14: 'Delhi Capitals'}


In [8]:
X = np.array(X)
y = np.array(y)

for i in range(len(X)):
    X[i][0] = all_teams[X[i][0]]
    X[i][1] = all_teams[X[i][1]]
    X[i][3] = all_teams[X[i][3]]

    y[i][0] = all_teams[y[i][0]]

print(X)



[[0 1 'field' 1]
 [2 3 'field' 3]
 [4 5 'field' 5]
 ...
 [0 14 'field' 14]
 [14 8 'field' 8]
 [2 8 'bat' 2]]


In [9]:
fb = {'field':0, 'bat':1}

for i in range (len(X)):
    X[i][2] = fb[X[i][2]]

print(fb)

{'field': 0, 'bat': 1}


In [10]:
for i in range(len(X)):
    if X[i][3] == X[i][0]:
        X[i][3] = 0
    else :
        X[i][3] = 1

print(['team1', 'team2', 'toss_decision', 'toss_winner'])
print(X)


['team1', 'team2', 'toss_decision', 'toss_winner']
[[0 1 0 1]
 [2 3 0 1]
 [4 5 0 1]
 ...
 [0 14 0 1]
 [14 8 0 1]
 [2 8 1 0]]


In [11]:
zeros = 0
for i in range(len(y)):
    if y[i][0] == X[i][1] :
        if zeros <= 740:
            zeros += 1
            y[i][0] = 1
        else:
            t = X[i][0]
            X[i][0] = X[i][1] 
            X[i][1] = t

            y[i][0] = 0
    else :
        y[i][0] = 0

print(y)

[[0]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [12]:
X = np.array(X, dtype='int32')
y = np.array(y, dtype='int32')

y = y.ravel()

print(np.unique(y, return_counts = True))


(array([0, 1]), array([763, 741], dtype=int64))


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.20)

In [32]:
from sklearn.svm import SVC
model1 = SVC().fit(X_train, y_train)
model1.score(X_test, y_test)

0.5614617940199336

In [33]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier().fit(X_train, y_train)
model2.score(X_test, y_test)

0.6013289036544851

In [48]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=300).fit(X, y)
model3.score(X_test, y_test)

0.7109634551495017

In [35]:

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_test, y_test)
classifier.score(X_test, y_test)

0.5681063122923588

In [17]:
test = np.array([1,2, 0, 0]).reshape(1,-1)
model1.predict(test)
model2.predict(test)
model3.predict(test)


array([1])

In [18]:
import pickle as pkl

with open('model.pkl', 'wb') as f:
    pkl.dump(model3, f)

with open('vocak.pkl', 'wb') as f:
    pkl.dump(encoded_teams, f)

with open('inv_vocab.pkl', 'wb') as f:
    pkl.dump(all_teams, f)